In [1]:
include("..\\src\\ImportanceSampling.jl")
using ImportanceSampling
using Distributions

In [2]:
srand(2016);

# use q

In [18]:
p = Normal(8, 1)
f!(r, x) = (r .= exp.(-x.^2/2))
μ_actual = exp(-16)/sqrt(2)

niters = 10_000
q = MixtureDistribution([Normal(4, sqrt(1/2)), Normal(2, 2)], [9, 1])
X = rand(q, 1, niters)

is = ImportanceSampler(f!, 1, q, p=p)
cvis = CvImportanceSampler(f!, 1, q, p=p, use_q=true)
;

In [19]:
update!(is, X=X[:, 1_001:end])
updateβ!(cvis, X=X[:, 1:1_000])
update!(cvis, X=X[:, 1_001:end])
;

In [20]:
μ_actual

7.957438516600104e-8

In [23]:
abs(mean(cvis) - μ_actual)/μ_actual

0.04382176690616389

In [22]:
abs(mean(cvis) - μ_actual), abs(mean(is) - μ_actual)

(3.4870901584458034e-9, 6.951338535200814e-11)

In [6]:
var(cvis), var(is)

(2.187929505805078e-48, 3.9754311543069156e-20)

# simple

In [7]:
g!s = [((r, x) -> r .= 2*x, [1.0])
    ((r, x) -> begin
        y = first(x)
        r[1] = 5*y.^2
        r[2] = 2*y.^3
        r
    end, [5/3, 1/2])]

f!(r, x) = r .= x.^2 - 3*x

niters = 10_000
q = Uniform()
X = rand(q, 1, niters)

cvis = CvImportanceSampler(f!, 1, q, g!s=g!s, w=((_)->1.0))
is = ImportanceSampler(f!, 1, q, w=((_)->1.0))

μ_actual = -7/6 
;

update!(is, X=X)
updateβ!(cvis, X=X[:, 1:1_000])
update!(cvis, X=X[:, 1_001:end])
;

In [8]:
coeffs(cvis)

3×1 Array{Float64,2}:
 -1.5        
  0.2        
 -1.64059e-13

In [9]:
mean(cvis), mean(is)

(-1.1666666666666665, -1.1605501287479578)

In [10]:
abs(mean(cvis) - μ_actual) , abs.(mean(is) - μ_actual)

(2.220446049250313e-16, 0.00611653791870892)

In [11]:
var(cvis), var(is)

(4.557350068424305e-33, 3.4226467239699256e-5)